In [14]:
import datetime as dt
import pyiqfeed as iq
import time
import pandas as pd
import os

In [28]:
def get_bars_before_settlement(start_datetime, end_datetime, 
                               bgn_flt, end_flt, ticker: str, 
                            bar_len: int, bar_unit: str, request_timeout: int):
    """Shows how to get interval bars."""
    hist_conn = iq.HistoryConn(name="pyiqfeed-Example-historical-bars")
    hist_listener = iq.VerboseIQFeedListener("History Bar Listener")
    hist_conn.add_listener(hist_listener)

    with iq.ConnConnector([hist_conn]) as connector:
        # look at conn.py for request_bars, request_bars_for_days and
        # request_bars_in_period for other ways to specify time periods etc
        try:
            bars = hist_conn.request_bars_in_period(ticker=ticker,
                                                    interval_len=bar_len,
                                                    interval_type=bar_unit,
                                                     bgn_flt=bgn_flt,
                                                     end_flt=end_flt,                                                  
                                                    bgn_prd=start_datetime,
                                                    end_prd=end_datetime,
                                                    timeout=request_timeout)
            return {'data':bars,'error_message':''}
        except (iq.NoDataError, iq.UnauthorizedError) as err:
            error_message = "No data returned because {0}".format(err)
            return {'data':[],'error_message':''}

# Downloading the Data

In [29]:
# define the name of the directory to be created
path_folder = r'data/' #os.getcwd() + '/data/'
if not os.path.isdir(path_folder):
    try:
        os.mkdir(path_folder)
    except OSError:
        print ("Creation of the directory %s failed" % path_folder)
    else:
        print ("Successfully created the directory %s " % path_folder)
else:
    print ("The directory %s already exists" % path_folder)

The directory data/ already exists


In [30]:
####LAUNCH IQFEED#####

In [31]:
df_symbols = pd.read_csv(r'df_symbols.csv', encoding='utf-8')

df_symbols['Settlement'] = pd.to_datetime(df_symbols['Settlement']).dt.time


df_symbols['start'] = df_symbols['Settlement'].apply(lambda x:
                (dt.datetime.combine(dt.date(1,1,1),x)-dt.timedelta(minutes = 5)).time())
df_symbols['end'] = df_symbols['Settlement']

In [32]:
bar_len=60
bar_unit='s'
request_timeout=60*5

start_datetime = pd.to_datetime('2018-01-01 00:00:00').to_pydatetime()
date_today = pd.to_datetime(datetime.datetime.today()).to_pydatetime()

#opening log
download_log = open("log.txt","a")

In [40]:
#loop over tickers and download
downloaded_tickers = []
for index, row in df_symbols.iterrows():
    ticker = row['Symbol']
    bgn_flt = row['start']
    end_flt = row['end']
    
    #requesting the data
    log_line = "Downloading symbol: %s..." % ticker
    log_line = log_line+str(start_datetime)+'_'+str(date_today)
    download_log.write(log_line+'\n')
    response = get_bars_before_settlement(start_datetime, date_today, bgn_flt, end_flt,
                            ticker, 
                         bar_len, bar_unit, request_timeout)
    data = pd.DataFrame(response['data'])
    download_log.write(response['error_message']+'\n')
    #saving the data
    if data.shape[0]>1:
        data.to_csv(path_folder + "%s.csv" % ticker, encoding = "utf-8",mode='a')
    time.sleep(1)
    downloaded_tickers.append(ticker)

download_log.close()        